# Applied Data Science Specialization Capstone

This notebook shows the process for completing the capstone project for the Applied Data Science Specialization Capstone course issued by Coursera in conjunction with IBM.

https://www.coursera.org/professional-certificates/ibm-data-science

In this assignment, the student must leverage location data using Foursquare API to solve a problem or get deeper insights into a neighborhood's reputation. 

[Create dataframe from Wikipedia table](#Segmenting-and-Clustering-Neighborhoods-in-Toronto)

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!') 

Hello Capstone Project Course!


In [12]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas import json_normalize

! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [4]:
CLIENT_ID = 'DEQ5TLK22PGVS00CRD4VC0EFMA5TOND42SPWYIS5DTJQQCCI' # Foursquare ID
CLIENT_SECRET = 'KZTNSCWYZT1MS5QIKTHHR1J25MRQF30K0E3JTMK3TJWTHWVL' # Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DEQ5TLK22PGVS00CRD4VC0EFMA5TOND42SPWYIS5DTJQQCCI
CLIENT_SECRET:KZTNSCWYZT1MS5QIKTHHR1J25MRQF30K0E3JTMK3TJWTHWVL


In [5]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


In [6]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


In [7]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=DEQ5TLK22PGVS00CRD4VC0EFMA5TOND42SPWYIS5DTJQQCCI&client_secret=KZTNSCWYZT1MS5QIKTHHR1J25MRQF30K0E3JTMK3TJWTHWVL&ll=40.7149555,-74.0153365&v=20180604&query=Italian&radius=500&limit=30'

In [8]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fa0b3e3dd96685c64e02292'},
 'response': {'venues': [{'id': '4fa862b3e4b0ebff2f749f06',
    'name': "Harry's Italian Pizza Bar",
    'location': {'address': '225 Murray St',
     'lat': 40.71521779064671,
     'lng': -74.01473940209351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71521779064671,
       'lng': -74.01473940209351},
      {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
     'distance': 58,
     'postalCode': '10282',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['225 Murray St',
      'New York, NY 10282',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-16043673

In [15]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4fa862b3e4b0ebff2f749f06,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1604367331,False,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",NaN
1,4f3232e219836c91c7bfde94,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1604367331,False,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN
2,3fd66200f964a520f4e41ee3,Ecco,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1604367331,False,124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,10007,US,New York,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,btwn Church St & W Broadway


# Segmenting and Clustering Neighborhoods in Toronto

Here we are going to read the html table using the read_html function from pandas, we also select the first data frame since this function returns multiple data frames.

In [19]:
df_toronto_pc = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

df_toronto_pc

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Here we make sure we remove the values with "Not assigned" as values in the Borough column

In [39]:
df_toronto_pc = df_toronto_pc.drop(df_toronto_pc[df_toronto_pc.Borough == 'Not assigned'].index)

df_toronto_pc


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


And then we make a count of the cells that have "Not assigned" values in the column Neighbourhood, as shown below we can see that we no longer have cells without values.

In [38]:
na_values = df_toronto_pc[df_toronto_pc.Neighbourhood == 'Not assigned']

na_values.count()

Postal Code      0
Borough          0
Neighbourhood    0
dtype: int64

In [41]:
df_toronto_pc.shape

(103, 3)